# (Imai, Kim, and Wang, 2023) Matching Methods for Causal Inference with Time-Series Cross-Sectional Data

*Some notes for the paper linked below. Let me know if you find any issues! I like to add my own notes here and there to help clarify any confusion.*

Updates:
- 11/23/23: took notes for the first third of the paper. The rest will be after Thanksgiving holidays.

https://imai.fas.harvard.edu/research/files/tscs.pdf

# Abstract

Matching methods improve the validity of causal inference by reducing model dependence and offering intuitive diagnostics. However, they are rarely used for analyzing time-series cross-sectional data, which this paper tries to solve.

>my note: If you are unsure of what TSCS is, the authors clear it up in the introduction.

# Introduction

TSCS data consist of a relatively large number of repeated measurements on the same units. In such data, each unit may receive the treatment multiple times and the timing of treatment administration may differ across units. Therefore, an overwelming number of social scientists use linear regression models with fixed effects, despite them relying heavily on parametric assumptions and offer few diagnostic tools and make it difficult to intuitively understand how counterfactual outcomes are estimated.

>my notes: For the rest of the introduction, the authors compare this methodology to other works by other influencial researchers. It appears that TSCS appears like a special case of panel data where the time periods $T$ is very large! For example, they note how models like MSM's by Robins doesn't work too well when there are an abnormal number of time periods.



# Motivating Applications

This methodology is inspired by two influencial studies:
1. Acemoglu et al. (2019), which examined the causal effect of democracy on economic development
2. Scheve and Stasavage (2012), which investigates whether war mobilization leads countries to introduce significant taxation of inherited wealth.

Both studies use linear regression models with fixed effects to estimat the causal effects of interest.

# Democracy and Economic Growth

Scholars have long debated whether democracy promotes economic development.

The authors of (Acemoglu et al., 2019) analyze an unbalanced TSCS dataset of a total of 184 countries over a half century from 1960 to 2010.

The main result uses the following dynamic linear regression model with country and year fixed effects,
$$Y_{it}=\alpha_i+\gamma_t+\beta X_{it}+\sum_{l=1}^{4}\{\rho_lY_{i,t-l}+\zeta_l^\top\mathbb{Z}_{i,t-l}\}+\epsilon_{it}$$
for $i=1,...,N$ and $t=5,...,T$ (the notation assumes a balanced panel for simplicity), where
- $Y_{it}$: logged real GDP per capita
- $X_{it}$: democracy indicator variable (at year $t$)
- 4 lagged outcome variables $Y_{i,t-l}$
- time-varying covariates $\mathbb{Z}_{i,t-l}$ as well as their lagged values.

These time varying covariates includes the log population, the log population below 16, above 64, net financial flow as a fraction of GDP, trade volume as a fraction of GDP and a binary measure of social unrest.

The choice of lags specifies how far back in time one needs to consider when adjusting for confounding factors.

Due to the nature of the model, the authors assume the following standard sequential exogeneity,
$$\mathbb{E}(\epsilon_{it}|Y_{i,t-1},...,Y_{i,1},X_{it},X_{i,t-1},...,X_{i1},\mathbb{Z}_{it},\mathbb{Z}_{i,t-1},...,\mathbb{Z}_{i1},\alpha_i,\gamma_t)=0$$
which implies that the error term is independent of past outcomes, current and past treatment and covariates.
>my note: While I'm not super familiar with econometrics theory, I think this is similar to conditional exchangeability where it's saying that once you control for all these covariates, the error term is now actually random, as in you have controlled for all systematic biases.

Because the OLS estimate of $\beta$ has an asymptotic bias of order $1/T$, Acemoglu et al. also fit the model using the generalized method of moments (GMM) estimation. The authors use the heteroskedasticity-robust standard errors.

>my note: the asymptotic bias is specified in a paper by (Nickell, 1981). It specifies how using the standard within-group estimator for dynamic models with fixed effects generates estimates which are inconsistent as the number of individuals tend to infinity if the number of time periods is kept fixed.

>my note: If you're curious of what it means to be a "dynamic" model, watch the video here: [YouTube Link](https://youtu.be/Ri6GM3CxYBI?si=7MGqxx0gVvo2xP3B). It essentially refers to the scenario where you include the assumption that the outcome $Y_{it}$ is dependent on past outcomes (e.g., $Y_{i,t-1}$), and when choosing to assume just one time period lag, it is a simplifying assumption called the *dynamic Markov model*.

>my note: In the above, they suggest GMM. Note that OLS is a special case of GMM, and the instrumental variables estimator is also related and is a special case of GMM. Read more here: [CrossValidated](https://stats.stackexchange.com/a/465883/391916).

The authors use 4 models, 2 with OLS, 2 with GMM, and conclude that in the year of democratization the GDP per capita increases more than 0.5%, a substantial effect given that democratization may have a long-term effect on economic growth.

# War and Taxation

While we skip the example mostly, the model is similar to the previous model but with a few changes - the model relies only on one lagged values of the treatment variable and time-varying covariates. Furthermore, they exclude lagged outcome variables. Thus, they are able to use OLS which requires the strict exogeneity assumption which is valid in this case because when you have past outcomes, you introduce lagged error terms and exogeneity may no longer be valid.

The authors use cluster-robust standard errors to account for auto-correlation within each country.

Recognizing the limitations of static models and yet wishing to avoid the bias of dynamic models, the authors of the paper on War and Taxation Scheve and Stasavage also fit a dynamic model (one lag) but with no country fixed effects (but still retaining the time fixed effects).

Then, OLS estimation can be employed and then panel-corrected standard errors are used to account for correlation across countries within a time period (Beck and Katz, 1995).

Scheve and Stasavage's results show that war has a positive estimated effect of several percentage points on inheritance taxation although the magnitude for contemporaneous effect in the dynamic models were much smaller.

## The Treatment Variation Plot

A drawback of most common methodological approaches to causal inference with TSCS data is that it is difficult to understand how they use observed data to estimate relevant counterfactual quantities.

With the *treatment variation plot*, which visualizes the variation of treatment cross space and time, which helps researchers build an intuitive about how comparison of treated and control observation can be made.

Please see example images in Figure 1 of the paper (linked in the beginning of the notebook).

# The Proposed Methodology

This section proposes a general matching method for causal inference with TSCS data which can be summarized as follows:

For each treated observation, researchers first find a set of control observations that have the identical treatment history up to the prespecified number of periods. This group of matched control observations is a *matched set*. Once a matched set is selected for each treated observation, we further refine it by adjusting for observed confounding via standard matching and weighting techniques. Finally, apply the DiD (differences-in-differences) estimator in order to account for an underlying time trend.

The authors then establish the connections to the linear fixed effects regression estimator and discuss covariate balance diagnostics and standard errors.

## Matching Estimators

For each unit $i=1,2,...,N$ at time $t=1,2,...,T$, we observe:
- outcome $Y_{it}$
- binary treatment indicator $X_{it}$
- vector of $K$ time-varying covariates $\mathbb{Z}_{it}$

In addition, within each time period,